In [12]:
import numpy as np
from numpy.linalg import inv

In [41]:
class CentralDifference:
    def __init__(self, dt, init_disp, init_R, k, m):
        self.dt = dt
        self.init_disp = init_disp
        self.k = k
        self.m = m
        
        self.velocity = np.zeros(init_disp.shape)
        self.accel = np.matmul(init_R, inv(self.m))
        
        self.a0 = 1 / dt ** 2
        self.a1 = 1 / 2 * dt
        self.a2 = 2 * self.a0
        self.a3 = 1 / self.a2
        
        self.em = inv(self.a0 * m)
        
        self._get_last(init_disp)
        
    def solve(self, U, R):
        el = R - np.matmul((self.k - self.a2 * self.m), U) - np.matmul(self.a0 * self.m, self.last_pos)
        nU = np.matmul(self.em, el)
        self.accel = self.a0 * (self.last_pos - 2 * U + nU)
        self.velocity = self.a1 * ((-1 * self.last_pos) + nU)
        self.last_pos = U
        return nU
    
    def _get_last(self, pos):
        self.last_pos = pos - self.dt * self.velocity + self.a3 * self.accel
        

In [ ]:
class LinearTetrahedral:
    def __init__(self)

In [45]:
init_disp = np.array([0, 0])
init_R = np.array([0, 10])
k = np.array([[6, -2], [-2, 4]])
m = np.array([[2, 0], [0, 1]])
cd = CentralDifference(0.28, init_disp, init_R, k, m)

U = np.array([0, 0])
R = np.array([0, 10])
for i in range(12):
    U = cd.solve(U, R)
    print(U)

[0.    0.392]
[0.0307328 1.4450688]
[0.16753064 2.83378293]
[0.48709385 4.14409153]
[1.01698937 5.01518935]
[1.70087983 5.25698772]
[2.39687119 4.9008927 ]
[2.91334844 4.16770713]
[3.07135437 3.36834164]
[2.77105574 2.77825258]
[2.0368198  2.53540505]
[1.0222996  2.60082784]
